In [5]:
import torch
from transformers import DistilBertTokenizer, DistilBertForTokenClassification
from transformers import pipeline

model_dir = 'DistilBert_conll03'

tokenizer = DistilBertTokenizer.from_pretrained(model_dir)
model = DistilBertForTokenClassification.from_pretrained(model_dir)

ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)

text = "Tamil Nadu is far from north-korea"
ner_results = ner_pipeline(text)

def aggregate_subwords(current_entity, final_entities):
    if current_entity:
        entity_word = ''.join(current_entity["words"]).replace('##', '')
        final_entities.append(f"Token: {entity_word}, Label: {current_entity['label']}")
        # final_entities.append(f"Token: {entity_word}"  # , Label: {current_entity['label']}
                              


current_entity = {}
final_entities = []

for entity in ner_results:
    if entity['word'].startswith('##'):
        # This is a subword of the current entity
        current_entity["words"].append(entity['word'])
    else:
        aggregate_subwords(current_entity, final_entities)
        current_entity = {
            "words": [entity['word']],
            "label": entity['entity']
        }

aggregate_subwords(current_entity, final_entities)

# Print the final entities
for entity in final_entities:
    print(entity)


Token: tamil, Label: B-LOC
Token: nadu, Label: I-LOC
Token: korea, Label: B-LOC


Token: kovai
CanonicalName: Coimbatore(kovai , blablabla)
Table: State

In [8]:
# import torch
# from transformers import DistilBertTokenizer, DistilBertForTokenClassification
# from fuzzywuzzy import process

# # Load DistilBERT model for NER
# # tokenizer = DistilBertTokenizer.from_pretrained('dbmdz/bert-large-cased-finetuned-conll03-english')
# # model = DistilBertForTokenClassification.from_pretrained('dbmdz/bert-large-cased-finetuned-conll03-english')

# distilbert-base-uncased

# # Example tables
# COUNTRIES = ["newzealand", "india", "usa"]
# CITIES = ["ahmedabad", "chennai", "newyork"]
# STATES = ["maharashtra", "texas", "california"]

# # Function to extract place names using NER
# def extract_place_names(sentence):
#     tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sentence)))
#     inputs = tokenizer.encode(sentence, return_tensors="pt")
    
#     outputs = model(inputs).logits
#     predictions = torch.argmax(outputs, dim=2)
    
#     # The ids 4 and 5 correspond to location entities in the 'dbmdz/bert-large-cased-finetuned-conll03-english' model
#     loc_tokens = [token for token, label in zip(tokens, predictions[0].tolist()) if label in [4, 5]]

#     return loc_tokens

# # Function to perform fuzzy matching and get canonical name
# def fuzzy_match(name, table):
#     matched_name, score = process.extractOne(name, table)
#     if score > 80:  # You can adjust this threshold
#         return matched_name
#     return None

# # Main function
# def geospatial_query_system(sentence):
#     place_names = extract_place_names(sentence)
    
#     results = []
#     for name in place_names:
#         canonical_name = None
#         source_table = None
        
#         for table, table_name in [(COUNTRIES, "Country"), (CITIES, "City"), (STATES, "State")]:
#             matched_name = fuzzy_match(name, table)
#             if matched_name:
#                 canonical_name = matched_name
#                 source_table = table_name
#                 break
        
#         if canonical_name:
#             results.append({
#                 "Token": name,
#                 "Canonical name": canonical_name,
#                 "Table": source_table
#             })
    
#     return results

# # Example
# sentence = "Which of the following saw highest average temperature in January, Maharashtra, Ahmedabad or entire New-Zealand?"
# print(geospatial_query_system(sentence))
